In [2]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.preprocessing import normalize

def gridSearch(model, parameters, x, y):
    scorer = make_scorer(mean_squared_error)
    model = GridSearchCV(model, param_grid=parameters, scoring=scorer)
    #model = grid.fit(x, y)
    print("Best score: ", model.best_score_)
    print("Best parameters for search: ", model.best_params_)
    return model

def randomSearch(model, parameters, x, y):
    scorer = make_scorer(mean_squared_error)
    model = RandomizedSearchCV(model, scoring=scorer, param_distributions=parameters)
    #model = rSearch.fit(x, y)
    print("Best score: ", model.best_score_)
    print("Best parameters: ", model.best_params_)
    return model

def plotResults(x, y, x_label, y_label, title):
    plt.plot(x, y)
    plt.title(title, fontsize=18)
    plt.xlabel(x_label, fontsize=16)
    plt.ylabel(y_label, fontsize=16)
    plt.grid('on')
    title+=".png"
    path = "graphs_beta"
    path += title
    plt.savefig(path)

In [4]:
#data processing here
header, raw_data = np.vsplit(data, [1])
raw_x, raw_y = np.hsplit(raw_data, [2])
raw_x_veg, raw_x_rain = np.hsplit(raw_x, [1])
data_train, data_test = train_test_split(raw_data, test_size=0.20)
x_test, y_test = np.hsplit(data_test, [2])
x_train, y_train = np.hsplit(data_train, [2])

In [ ]:
plotResults(raw_x_veg, raw_y, "Prior Vegitation", "Vegitation Lost", "Prior Vegitation vs Vegitation Lost")
plotResults(raw_x_rain, raw_y, "Precipitation", "Vegitation Lost", "Precipitation vs Vegitation Lost")

In [ ]:
cValues = np.logspace(-3, 3, 7, dtype=float)
parameters = {"c": cValues}
svr = SVR(kernel="linear")
svrLinearModel = gridSearch(svr, parameters, x_train, y_train)

In [ ]:
print("Best parameters for svr")
cValues = np.logspace(-3, 3, 7, dtype=float)
parameters = {"c": cValues}
svr = SVR(kernel="poly")
svrPolyModel = gridSearch(svr, parameters, x_train, y_train)

In [ ]:
mean_scores = svrLinearModel.cv_results_['mean_test_score']
mean_scores = np.delete(mean_scores, [0,1])
x_cValues = np.delete(cValues, [0,1])
plotResults(x_cValues, mean_scores, "c values", "Scores", "Linear: c values vs mean score")